# Add crossings

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import shapely.geometry as sg
import geopandas as gpd
from shapely import wkt
import momepy
import networkx as nx
from branca.element import Template, MacroElement
from tqdm.notebook import tqdm
import branca.colormap as cm

import folium

import crossing_utils
import plot_utils

import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

### Import walking and road network 

### NOTE: If the cf.output_road_network file does not exist yet, run the create_road_network.ipynb notebook before running this notebook

In [ ]:
# Get basic pedestrian network with widths
CRS = st.CRS

gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=CRS)
graph_network = momepy.gdf_to_nx(gdf_network, approach='primal')
cc_list = [x for y in zip(gdf_network['sidewalk_id'], gdf_network['sidewalk_id']) for x in y]
gdf_network_nodes = gpd.GeoDataFrame(geometry = gdf_network.boundary.explode(index_parts=True), crs=CRS)
gdf_network_nodes['x'], gdf_network_nodes['y'] = gdf_network_nodes.geometry.x, gdf_network_nodes.geometry.y
gdf_network_nodes['cc'] = cc_list

gdf_road_network = gpd.read_file(cf.output_road_network).to_crs(crs=CRS)

# 1. Create crossings with curb heights

### Import curb segments

In [ ]:
# Import curb heights
ch_crossing_features = cf.output_curb_heigts
df_ch = pd.read_csv(ch_crossing_features)
df_ch['geometry'] = df_ch['line_segm'].apply(wkt.loads)
gdf_ch = gpd.GeoDataFrame(df_ch, crs=CRS)
graph_ch = momepy.gdf_to_nx(gdf_ch, approach='primal')

### Select curb segments to keep


In [ ]:
# Create network and crossing feature dataframes

# TODO 
# Enable setting max_height >= 10cm (e.g., by generating compute with more RAM or compute crossings in smaller areas)

# SET THIS VALUE
max_height = 0.04 # in meter

gdf_ch.dropna(subset=['curb_height'])
gdf_ch = gdf_ch.loc[(gdf_ch['curb_height'] <= max_height)]
gdf_ch['centroid'] = gdf_ch['geometry'].centroid
gdf_ch.reset_index(drop=True, inplace=True)

### Determine connected components of curb edges 
#### These are used as a restrictive rule when creating possible crossing edges (i.e., crosing edges should connect different curbs)

In [ ]:
# Calculate connected components for each edge
# Option 1: From curb height edges
# Option 2: From sidewalk edges
# Option 1 results in more connected components since the curb height edges are more fragmentated (i.e., not all parts of sidewalk egde have height information)

# Calculate using the connected components retrieved from the curb height edges
cc_sidewalks = list(nx.connected_components(graph_ch))
nodes_in_edge = [line.coords[0] for line in gdf_ch['geometry'].to_list()]
cc_ids = [i for node in nodes_in_edge for i in range(len(cc_sidewalks)) if node in cc_sidewalks[i]]
gdf_ch['cc_from_curb_edges'] = cc_ids

# Calculate using the connected components retrieved from sidewalk edges from bgt (from road_curb_segments.ipynb notebook)
gdf_ch['cc_from_sidewalk_edges'] = gdf_ch.groupby('overarching_line_segm').ngroup()

### Calculate curb ramp crosswalks 

In [ ]:
# TODO
# Include angle of intersection between crosswalk edge and road centerline as critera

# Get coordinates of crossing feature nodes
gdf_ch['geometry'] = gdf_ch['centroid']
gdf_ch['x'], gdf_ch['y'] = gdf_ch.geometry.x, gdf_ch.geometry.y

# Connect curb height nodes to curb height nodes
min_dist = 0
max_dist = 11
max_connections = 1
gdf_curb_edges = crossing_utils.get_crossing_edges_from_curb_heights(gdf_ch, gdf_ch, gdf_road_network, 
                    min_dist=min_dist, max_dist=max_dist, max_connections=max_connections, cc_column='cc_from_sidewalk_edges', crs=CRS)
print('Number of new edges from curb node to curb node:', len(gdf_curb_edges))

### Connect curb ramp crossings to walking network

In [ ]:
# Current heuristic: Connect outer points of crossing edge to closest network nodes within max_dis.
# Network nodes cannot be part of the same sidewalk
# TODO make this neater

# Get outer nodes of crossing edges and connect to network
gdf_curb_edge_nodes = gpd.GeoDataFrame(geometry=gdf_curb_edges['geometry'].boundary.explode(index_parts=True), crs=CRS).reset_index()
gdf_curb_edge_nodes['x'], gdf_curb_edge_nodes['y'] = gdf_curb_edge_nodes.geometry.x, gdf_curb_edge_nodes.geometry.y
groups = gdf_curb_edge_nodes.groupby(np.arange(len(gdf_curb_edge_nodes.index))//2)

# Loop over possible crossing edges seperately and connect to walking network
gdf_crossings = gpd.GeoDataFrame()
network_to_network = True # Set False if you wish to connect network to curb instead of network to network 

for (idx, frame) in tqdm(groups):
    sub_gdf = crossing_utils.connect_curb_crossing_edge(frame, gdf_network_nodes, graph_network, max_dist=4, max_connections=1, crs=CRS, network_to_network=network_to_network)
    gdf_crossings = pd.concat([gdf_crossings, sub_gdf])

print('Connected valid crossings')

### create map and visualize crossings on map

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=plot_utils.generate_map_params(satelite=satelite),
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add colormap
cmp = cm.linear.RdYlGn_11.colors
cmp = list(reversed(cmp))
colormap = cm.LinearColormap(colors=cmp, vmin=0, vmax=0.2, caption='Curb height (m)')
colormap.add_to(map)

# Add network and new edges
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_crossings, style_function=lambda x: {"color": "red", "weight": 4}).add_to(map)

# Plot map
template = plot_utils.gen_template_curbs()
macro = MacroElement()
macro._template = Template(template)
map.get_root().add_child(macro)
map

### Save new edges

In [ ]:
gdf_crossings.to_file(cf.output_curb_crossings_base + '_max_height_{}.gpkg'.format(max_height), driver='GPKG')